In [1]:
import pandas as pd
import numpy as np

from utils.data_utils import DataUtils

In [2]:
pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_colwidth", 100)

----

#### Download data

In [3]:
DataUtils.download_data("sql/select_comments.sql", "comments")

In [5]:
DataUtils.download_data("sql/select_tags.sql", "tags")

In [6]:
comments = pd.read_parquet("data/initial_data/comments.parquet")
comments.query("not comment.isna() & not place_id.isna()", inplace=True)

In [7]:
tags = pd.read_parquet("data/initial_data/tags.parquet")

In [8]:
df = pd.DataFrame().from_records(comments.comment.values)

In [11]:
df.insert(0, "place_id", comments.place_id.values)

In [12]:
df = df.merge(tags, left_on="place_id", right_on="place_id", how="left")

In [13]:
df.drop("id", axis=1, inplace=True)

In [14]:
df.to_parquet("data/intermediate_data/comments.parquet")

-----

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("data/intermediate_data/comments.parquet")

In [3]:
df.sample(5)

,place_id,review_body,review_date,review_likes,review_rating,review_tag,review_title,tag
2780,29,"Памятник. прямо скажем, бредовый. Сделан тяп-л...",31 июля 2014 г.,1.0,2.0,"[Путешествие в одиночку, RU]",Тяп-ляп,Памятники и статуи
3931,119,Отличное место. Помню как возмущались жители д...,21 октября 2014 г.,0.0,5.0,"[Семейный отдых, RU]",Отличное место,Другие виды активного отдыха на открытом воздухе
1470,3,Провинциальная попытка сделать интересное мест...,30 января 2013 г.,1.0,3.0,"[Семейный отдых, RU]",Странное впечатление,Парки
2849,31,"Хочу сказать для гостей города, что Тюменский ...",19 июля 2015 г.,1.0,5.0,"[Семейный отдых, RU]",Красивое современное здание Тюменского театра ...,Театры
1914,8,"Хорошо отремонтированный, чистый парк с беспла...",31 августа 2014 г.,1.0,5.0,"[Бизнес, RU]",Замечательный парк,Парки


In [4]:
test = df.review_body[0]
test1 = df.review_body[2]
test2 = df.review_body[3]
test3 = df.review_body[124]
test4 = df.review_body[412]

print(test, "\n")
print(test1, "\n")
print(test2, "\n")
print(test3, "\n")

Красивая набережная с барельефами отражающими историю города Тюмень Красивые виды Зимой строят специальную горку для фристайла и сноуборда Можно посмотреть соревнования а можно просто погулять 

Отличное место для отдыха, прогулок и даже знакомства с историей города и Сибири. Очень круто, что скульптурные композиции, расположенные вдоль прогулочного маршрута, знакомят вас с историей региона, города, выдающихся личностей, а не как это сейчас популярно делать во многих городах - ни к чему не относящиеся, и ни о чем не говорящие, скульптуры а-ля городовой, кот/пёс/лис/конь на удачу и т.п.
Обязательно надо прогуляться по набережной днем, а затем в вечернее время, когда включается архитектурно-художественная подсветка. Идея сделать мост не просто красиво, а в форме струги, придаёт масштаб всей концепции историзма набережной Туры. 

Красивая набережная, можно покататься на самокате, гуляет много народу, мост влюбленных вечером красиво светится 

Отличное место во всех смыслах! Погулять пароч

In [5]:
# from models import mt5_ml_xls
from models import roberta

/home/dmitry/.cache/pypoetry/virtualenvs/lewart-tags-model-dMjvlOT--py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from models import roberta
model = roberta()

In [7]:
# TODO() кластеризация по эмбендингам

In [31]:
review = test3
proba = 0.0
questions = [
    "Чем заняться?", 
    "Какой?", 
    "Когда?", 
    "Какие ощущения", 
    "Почему?", 
    "Для кого?", 
    "С кем?",
    "Что?",
    "Кому?",
    "Для чего?"
]

In [32]:
from pprint import pprint
pprint(review)
print("\n")
for question in questions:
    print(question, ": ", model.get_tags(review, question, proba=proba))

('Отличное место во всех смыслах! Погулять парочкам, покататься на '
 'велосипедах, заняться пробежкой, прогуляться с родителями, пообщаться с '
 'друзьями... И так далее... И это всё она, набережная Тюмени, прекрасна в '
 'любое время суток, приезжайте и убедитесь сами 😊')


Чем заняться? :  [array(['пробежка'], dtype='<U8'), (0.742, 0.874)]
Какой? :  [array(['отличное место'], dtype='<U14'), (0.473, 0.249)]
Когда? :  [array(['любое время суток'], dtype='<U17'), (0.899, 0.866)]
Какие ощущения :  []
Почему? :  []
Для кого? :  [array(['погулять парочкам'], dtype='<U17'), (0.875, 0.287)]
С кем? :  [array(['друг'], dtype='<U4'), (0.346, 0.637)]
Что? :  [array(['набережная тюмени'], dtype='<U17'), (0.742, 0.478)]
Кому? :  [array(['погулять парочкам'], dtype='<U17'), (0.877, 0.476)]
Для чего? :  []


## Score the data

In [1]:
import pandas as pd
df = pd.read_parquet("data/intermediate_data/comments.parquet")

In [2]:
from models import roberta
model = roberta()

/home/dmitry/.cache/pypoetry/virtualenvs/lewart-tags-model-dMjvlOT--py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df.head()

,place_id,review_body,review_date,review_likes,review_rating,review_tag,review_title,tag
0,1,Красивая набережная с барельефами отражающими ...,27 февраля 2023 г.,0.0,5.0,[RU],Интересно прогуляться,Пирсы и пляжные променады
1,1,Очень приятно прогуляться по набережной . Вече...,12 декабря 2022 г.,0.0,5.0,[RU],Место для романтических поогулок,Пирсы и пляжные променады
2,1,"Отличное место для отдыха, прогулок и даже зна...",16 ноября 2022 г.,0.0,5.0,[RU],Концептуально-выдержанная идея вплоть до мелочей,Пирсы и пляжные променады
3,1,"Красивая набережная, можно покататься на самок...",2 сентября 2022 г.,1.0,5.0,[RU],Набережная в Тюмени,Пирсы и пляжные променады
4,1,Убрать всех приставучих предпринимателей! Приш...,27 июля 2022 г.,1.0,1.0,"[С друзьями, RU]",Испортили хорошее место торгашами.,Пирсы и пляжные променады


In [4]:
proba = 0.0
questions = [
    "Чем заняться?", 
    "Какой?", 
    "Когда?", 
    "Какие ощущения", 
    "Почему?", 
    "Для кого?", 
    "С кем?",
    "Что?",
    "Кому?",
    "Для чего?"
]
for question in questions:
    df[question] = df.review_body.apply(model.get_tags, args=(question, 0.0))

Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors


In [8]:
questions = df.iloc[:,8:].copy()

In [59]:
def choose_list(list_data, col):
    try:
        return list(list_data[col])
    except:
        pass


questions_data = pd.DataFrame()
for question in questions.columns:
    questions_data[question+"_data"] = questions[question].apply(choose_list, args=[0])
    questions_data[question+"_proba"] = questions[question].apply(choose_list, args=[1])

In [60]:
data = df.iloc[:,:8].join(questions_data)

In [62]:
data.to_parquet("./data/prepared_data/df_tagged.parquet")

In [63]:
data.head()

,Unnamed: 0,place_id,review_body,review_date,review_likes,review_rating,review_tag,review_title,Чем заняться?_data,Чем заняться?_proba,...,Для кого?_data,Для кого?_proba,С кем?_data,С кем?_proba,Что?_data,Что?_proba,Кому?_data,Кому?_proba,Для чего?_data,Для чего?_proba
0,0,1,Красивая набережная с барельефами отражающими ...,27 февраля 2023 г.,0.0,5.0,['RU'],Интересно прогуляться,"[посмотреть, соревнования, погулять]","[0.369, 0.701]",...,[фристайла сноуборда],"[0.613, 0.797]",None,None,[красивая набережная барельефами отражающими и...,"[0.426, 0.307]","[посмотреть, соревнования, погулять]","[0.448, 0.234]",None,None
1,1,1,Очень приятно прогуляться по набережной . Вече...,12 декабря 2022 г.,0.0,5.0,['RU'],Место для романтических поогулок,[детям классно погонять на самокатах],"[0.418, 0.661]",...,[ребёнок],"[0.957, 0.479]",[детям классно погонять на самокатах],"[0.888, 0.48]",None,None,[ребёнок],"[0.978, 0.69]",[детям классно погонять на самокатах],"[0.617, 0.504]"
2,2,1,"Отличное место для отдыха, прогулок и даже зна...",16 ноября 2022 г.,0.0,5.0,['RU'],Концептуально-выдержанная идея вплоть до мелочей,"[отдых, прогулка]","[0.264, 0.461]",...,None,None,None,None,None,None,"[отдых, прогулка]","[0.537, 0.32]",None,None
3,3,1,"Красивая набережная, можно покататься на самок...",2 сентября 2022 г.,1.0,5.0,['RU'],Набережная в Тюмени,[покататься на самокате],"[0.514, 0.844]",...,[гуляет народу],"[0.712, 0.821]",[гуляет народу],"[0.67, 0.494]",[красивая набережная],"[0.906, 0.532]",[гуляет народу],"[0.664, 0.888]",[покататься на самокате],"[0.527, 0.385]"
4,4,1,Убрать всех приставучих предпринимателей! Приш...,27 июля 2022 г.,1.0,1.0,['С друзьями' 'RU'],Испортили хорошее место торгашами.,None,None,...,[убрать приставучих предпринимателей],"[0.853, 0.737]",[мадам питоном],"[0.494, 0.774]",[убрать приставучих предпринимателей],"[0.573, 0.531]",[убрать приставучих предпринимателей],"[0.725, 0.7]",[убрать приставучих предпринимателей],"[0.962, 0.678]"
